# Bifurcation equations {#sec-20230102153124}

This notebook uses $\LaTeX\newcommand{\D}{\mathrm{d}}\newcommand{\E}{\mathcal{E}}\newcommand{\order}[2][1]{#2^{(#1)}}\newcommand{\reals}{\mathbb{R}}$ custom macros.

In this chapter, the bifurcation analysis of the perfect system is performed symbolically. The starting point is the symbolic expression of the energy $(u, \lambda) \mapsto \E(u, \lambda)$ that was derived in @sec-20230208081549.

To begin with, we perform the relevant initializations and retrieve the expression of the energy.

In [1]:
import itertools

from lsk import *
from sympy import *

In [2]:
#VERBOSE = False

#E_ij_dot, E_il_dot = symbols(r"\dot{E}_{ij} \dot{E}_{il}")
#E_il_ddot = symbols(r"\ddot{E}_{il}")
#E_ijk, E_ijl = symbols(r"E_{ijk} E_{ijl}")
#E_ijk_dot = symbols(r"\dot{E}_{ijk}")
#E_ijkl = symbols(r"E_{ijkl}")

#t, λ = symbols(r"t \lambda")
#u, u_hat, w_hat = symbols(r"u \hat{u} \hat{w}")

In [3]:
t, u_hat, w_hat = symbols(r"t \hat{u} \hat{w}")

In [4]:
# Naming conventions 
# - Instances of the `IndexedBase` class end with `_`

λ_ = IndexedBase(r"\lambda")
ξ_ = IndexedBase(r"\xi")

v_ = IndexedBase("v")
w_ = IndexedBase("w")

E_ = IndexedBase("E")
E_dot_ = IndexedBase("\dot{E}")
E_ddot_ = IndexedBase("\ddot{E}")

In [5]:
i = Idx("i")
j = Idx("j")
k = Idx("k")
l = Idx("l")

In [6]:
# This dicts can be passed to `subs`

w_is_symmetric = {
    w_[j, i]: w_[i, j],
    w_[k, i]: w_[i, k],
    w_[k, j]: w_[j, k],
    w_[l, i]: w_[i, l],
    w_[l, j]: w_[j, l],
    w_[l, k]: w_[k, l]
}

λ_is_symmetric = {
    λ_[j, i]: λ_[i, j],
    λ_[k, i]: λ_[i, k],
    λ_[k, j]: λ_[j, k],
    λ_[l, i]: λ_[i, l],
    λ_[l, j]: λ_[j, l],
    λ_[l, k]: λ_[k, l]
}

In [7]:
__index_number = itertools.count()

def new_index():
    return Idx(r"i_{" + str(next(__index_number)) + "}")

In [8]:
def symmetrize(expr):
    if not expr.is_Mul:
        raise ValueError()
    struct = get_contraction_structure(expr)
    indices = next(iter(struct))
    if len(indices) == 3:
        j, k, l = indices
        # Assign temporary indices
        jj = new_index()
        kk = new_index()
        ll = new_index()
        expr = expr.subs({j: jj, k: kk, l: ll})
        return (expr.subs({jj: j, kk: k, ll: l})
                + expr.subs({jj: j, kk: l, ll: k})
                + expr.subs({jj: k, kk: l, ll: j})
                + expr.subs({jj: k, kk: j, ll: l})
                + expr.subs({jj: l, kk: j, ll: k})
                + expr.subs({jj: l, kk: k, ll: j})) / 6
    else:
        raise ValueError()

In [9]:
def standardize_indices(expr):
    if not expr.is_Mul:
        raise ValueError()
    struct = get_contraction_structure(expr)
    indices = next(iter(struct))
    if len(indices) == 3:
        j, k, l = indices
        return expr.subs({j: Idx("j"), k: Idx("k"), l: Idx("l")})
    else:
        raise ValueError()

In [10]:
u_star = create_u_star()
E = create_E().expand()

In [11]:
#| code-fold: true
display_latex_equation(r"u^\star(\lambda)", u_star)
display_latex_equation(r"\E(u, \lambda)", E)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Outline of the strategy

In this section, we show that, besides the fundamental branch $u^\ast(\lambda)$, other (bifurcated) equilibrium branches may pass through the critical point $(u_0, \lambda_0)$. The starting point is the characterization of an equilibrium by the stationarity of the energy, which defines all equilibrium branches as implicit functions, which can be expanded with respect to some perturbation parameter.

The approach adopted here relies on the Lyapunov–Schmidt decomposition (see below). Note that other approaches are possible, see e.g. @chak2018.

For a given value of $\lambda$ (close to the critical value $\lambda_0$), we seek an equilibrium point $u$ of the
system, such that

$$
\E_{,u}(u, \lambda; \hat{u}) = 0 \quad \text{for all} \quad \hat{u} \in U.
$$ {#eq-20230102025737}

In [12]:
res = (create_E_u() * u_hat).expand()

In [13]:
#| code-fold: true
display_latex_equation(res, 0)

<IPython.core.display.Math object>

for all $\hat{u} \in U$.

The equilibrium state $u$ is projected onto the orthogonal subspaces $V$ and $W$.
$$
u = u^\ast(\lambda) + \xi_i \, v_i + w, \quad \text{with} \quad w \in W.
$$ {#eq-20220902174235}

It follows from the orthogonality of $V$ and $W$ that $\langle v_i, w \rangle = 0$ for all $i=1, \ldots, m$. Analysis of the bifurcated branches therefore reduces to finding $\xi_1, \ldots, \xi_m \in \reals$ and $w \in W$, such that
$$
\E_{,u}[u^\ast(\lambda) + \xi_i \, v_i + w, \lambda; \hat{u}] = 0 \quad \text{for all} \quad \hat{u} \in U.
$$ {#eq-20230107164346}

The method proceeds in three steps

1. Eq. (@eq-20230107164346) is first expressed for $\hat{u} \in W$ which leads to a definition of $w$ as an implicit function of $\xi_1, \ldots, \xi_m$ and $\lambda$ (see @sec-20230102030010),
2. Eq. (@eq-20230107164346) is then expressed for $\hat{u} \in V$ which leads to a definition of $\lambda$ as an implicit function of $\xi_1, \ldots, \xi_m$ (see @sec-20230102030125),
3. finally, a parametrization $\eta$ of $\xi_1, \ldots, \xi_m$ is introduced and the Taylor expansion of $u$ and $\lambda$ with respect to $\eta$ is derived (see @sec-20230102030224).

## Elimination of $w$ {#sec-20230102030010}

We first test Eq. (@eq-20230102025737) with $\hat{w} \in W$
$$
\E_{,u}[u^\ast(\lambda) + \xi_i \, v_i + w, \lambda; \hat{w}] = 0, \quad \text{for all} \quad \hat{w} \in W.
$$ {#eq-20220825143616}

This defines $w$ as an implicit function of $\xi_1, \ldots, \xi_m$ and $\lambda$. The following asymptotic expansion about $\xi_1 = \cdots = \xi_m = 0$ and $\lambda = \lambda_0$ is postulated
$$
\begin{aligned}
w(\xi_1, \ldots, \xi_m, \lambda) ={} & \xi_i \, w_i + \bigl( \lambda - \lambda_0 \bigr) \, w_\lambda + \tfrac{1}{2} \xi_i \, \xi_j \, w_{ij}\\
&+ \tfrac{1}{2} \bigl( \lambda - \lambda_0 \bigr)^2 w_{\lambda\lambda} + \xi_i \, \bigl( \lambda - \lambda_0 \bigr) w_{i\lambda} + \cdots,
\end{aligned}
$$ {#eq-20230107172048}
where $w_i$, $w_\lambda$, $w_{ij}$, $w_{\lambda\lambda}$ and $w_{i\lambda}$ are elements of $W$ to be defined.

In what follows, Eq. (@eq-20230107172048) is plugged into Eq. (@eq-20220825143616), which then delivers an asymptotic expansion of the stationarity equation. We will nullify the terms of order up to 2 in $\xi_i$ and $\lambda$.

It is observed that the residual $\E_{,u}$ involves $\E_3(u, u, \hat{u})$ and $\E_4(u, u, u, \hat{u})$. Since all terms of $u$ are of order 1 or more in $\xi_i$ and $\lambda$, $\E_4(u, u, u, \hat{u})$ will not be considered below. However $\E_3(u, u, \hat{u})$ will need to be properly defined index-wise. Therefore, we need to defined two equivalent expressions of $u$ with two different set of indices.

In [14]:
create_λ = default_λ_fun

def create_w():
    i = new_index()
    j = new_index()
    k = new_index()
    l = new_index()
    _λ = Idx("\lambda")
    return (t * ξ_[i] * w_[i] + create_λ() * w_[_λ] 
            + (t**2 * ξ_[j] * ξ_[k] * w_[j, k]
               + 2 * create_λ() * t * ξ_[l] * w_[l, _λ]
               + create_λ() * create_λ() * w_[_λ, _λ]) / 2)
    
def create_u():
    i = new_index()
    return (create_u_star(create_λ) + t * ξ_[i] * v_[i] + create_w())

In [15]:
res_w = expand(create_E_u(u_fun=create_u) * w_hat)

We can then nullify the various coefficients of this polynomial expansion, starting with the terms of order 1 in $\xi_i$, which delivers the following variational problem

In [16]:
λ = create_λ()
Eq(res_w.coeff(t, 1).subs(λ, 0), 0)

Eq(\E_2*\hat{w}*\xi[i_{0}]*v[i_{0}] + \E_2*\hat{w}*\xi[i_{1}]*w[i_{1}], 0)

for all $\hat{w} \in W$. Since $v_i \in V$, we have: $\E_2(v_i, \bullet) = 0$ and the above variational problem reduces to

$$
\E_2(w_i, \hat{w}) = 0 \quad \text{for all} \quad \hat{w} \in W.
$$

As argued in @sec-20230107173921, this leads to $w_i = 0$.

Similarly, we find the term or of order 1 in $\lambda$, and evaluate it at $\xi_1 = \cdots = \xi_m = 0$. Unfortunately, substitutions do not work with indexed quantities. Since all occurences of $\xi_i$ have been scaled by $t$ so as to keep track of the order of each term, a workaround is to set $t$ to 0. This delivers the following variational problem

In [17]:
res_w.coeff(λ, 1).subs(t, 0)

\E_2*\hat{w}*w[\lambda]

for all $\hat{w} \in W$. The same argument leads to $w_\lambda = 0$.

Similarly again, for the term of order 2 in $\lambda$ (remembering that $w_\lambda = 0$)

In [18]:
(eq := Eq(res_w.coeff(λ, 2).subs({
    t: 0,
    w_[Idx(r"\lambda")]: 0
}), 0))

Eq(\E_2*\hat{w}*w[\lambda, \lambda]/2, 0)

for all $\hat{w} \in W$ and $w_{\lambda\lambda} = 0$.

To sum up, we have found so far that $w_i = w_\lambda = w_{\lambda\lambda} = 0$. At this point, it is interesting to override the function `_w()`, taking into account all these simplifications. From now on, $w_{ij}$ and $w_{i\lambda}$ will be defined as two different `IndexedBase` objects.

In [19]:
def create_w():
    j = new_index()
    k = new_index()
    l = new_index()
    _λ = Idx(r"\lambda")
    return t**2 * ξ_[j] * ξ_[k] * w_[j, k] / 2 + create_λ() * t * ξ_[l] * w_[l, _λ]

We can then re-evaluate the residual

In [20]:
res_w = expand(create_E_u(u_fun=create_u) * w_hat)

The $\xi_i \, \xi_j$ terms deliver the following variational problem

In [21]:
lhs = res_w.coeff(t, 2).subs(λ, 0).expand()
Eq(lhs, 0)

Eq(\E_2*\hat{w}*\xi[i_{51}]*\xi[i_{52}]*w[i_{51}, i_{52}]/2 + \E_3*\hat{w}*\xi[i_{54}]*\xi[i_{58}]*v[i_{54}]*v[i_{58}]/2, 0)

for all $\hat{w} \in W$ and $\xi_1, \ldots, \xi_m \in \reals$ (small enough). This variational problem  should be understood as

$$
\xi_i \, \xi_j \, \bigl[ \E_2(w_{ij}, \hat{w}) + \E_3(v_i, v_j, \hat{w}) \bigr] = 0
\quad \text{for all} \quad \hat{w} \in W.
$$

for all $\hat{w} \in W$ and $\xi_1, \ldots, \xi_m$ *small enough*. Since the above expression is homogeneous in $\xi_i$, it must in fact vanish *for all* $\xi_1, \ldots, \xi_m$. The resulting variational problem then coincides with (@eq-20230107180410). The $\xi_i \, \xi_j$ term of the asymptotic expansion of $w$ is therefore exactly $w_{ij}$ defined in @sec-20230402152824.

Finally, the $\xi_i \, \lambda$ terms deliver the following variational problem

In [22]:
lhs = res_w.coeff(t, 1).coeff(λ, 1).subs(λ, 0).expand()
Eq(lhs, 0)

Eq(\E_2*\hat{w}*\xi[i_{53}]*w[i_{53}, \lambda] + \E_3*\dot{u}_0*\hat{w}*\xi[i_{54}]*v[i_{54}]/2 + \E_3*\dot{u}_0*\hat{w}*\xi[i_{58}]*v[i_{58}]/2 - \E_3*\dot{u}_0*\hat{w}*\xi[i_{62}]*v[i_{62}] + \dot{\E}_2*\hat{w}*\xi[i_{62}]*v[i_{62}], 0)

In the above expression, each term contains only one summation index, which is renamed $i$. This transformation delivers the following variational problem.

In [23]:
terms = lhs.args
lhs = 0

for term in terms:
    d = get_contraction_structure(term)
    (i_,), = d.keys()
    lhs += term.subs(i_, i)

In [24]:
#| code-fold: true
display_latex_equation(lhs, 0)

<IPython.core.display.Math object>

for all $\hat{w} \in W$ and $\xi_1, \ldots, \xi_m \in \reals$. We recognize the variational problem (@eq-20230107180501). In other words, the $\xi_i \, \lambda$ term of the asymptotic expansion of $w$ is $w_{i\lambda}$ defined in @sec-20230402152824.

So far, we have found the following expansion of $w$

$$
w(\xi_1, \ldots, \xi_m, \lambda) = \tfrac{1}{2} \xi_i \, \xi_j \, w_{ij} + \xi_i \, \bigl( \lambda - \lambda_0 \bigr) w_{i\lambda} + \cdots,
$$

where $w_{ij}$ and $w_{i\lambda}$ are defined by Eqs. (@eq-20230107180410) and (@eq-20230107180501), respectively.

## Elimination of $\lambda$ {#sec-20230102030125}

We now test Eq. (@eq-20230102025737) with $\hat{u} = v_i$ ($i = 1, \ldots, m$)

$$
\E_{,u}[u^\ast(\lambda) + \xi_j \, v_j + w(\xi_1, \ldots, \xi_m, \lambda), \lambda; v_i] = 0, \quad \text{for all} \quad i = 1, \ldots, m.
$$ {#eq-20230110092313}

This now defines $\lambda$ as an implicit function of $\xi_1, \ldots, \xi_m$. The following asymptotic expansion
about $\xi_1 = \cdots = \xi_m = 0$ is postulated

$$
\lambda(\xi_1, \ldots, \xi_m) = \xi_i \, \lambda_i + \tfrac{1}{2} \, \xi_i \, \xi_j \, \lambda_{ij} + \ldots
$$ {#eq-20230110091031}

where $\lambda_i$ and $\lambda_{ij}$ are scalar coefficients to be defined below.

We first override the function `create_λ()` that creates (with unique indices) the SymPy expression for the asymptotic expansions of  $(\xi_1, \ldots, \xi_m) \mapsto \lambda(\xi_1, \ldots, \xi_m)$ according to Eq. (@eq-20230110091031). Since the functions `create_u()` and `create_w()` call `_λ()` internally, there is no need to redefine them to implement the asymptotic expansions of $(\xi_1, \ldots, \xi_m, \lambda) \mapsto w(\xi_1, \ldots, \xi_m, \lambda)$ and $(\xi_1, \ldots, \xi_m) \mapsto u(\xi_1, \ldots, \xi_m)$ according to Eqs. (@eq-20230107172048) and (@eq-20220902174235).

Although $\lambda$ is expanded to second-order in $\xi_i$, we will consider third-order terms of the energy below. Therefore, we must make sure that the third-order term of the residual does not involve the third-order term of $\lambda$. To do so, we define the third-order term of the expansion of $\lambda$ as a symbol, $\order[3]{\lambda}$. We will check later on that this term disappears from the third-order expansion of the energy. We also include the $\mathcal{O}(t^4)$ term, that will speed up subsequent calculations (by cancelling many terms).

In [25]:
λ3 = Symbol(r"\order[3]{\lambda}")

def create_λ():
    i = new_index()
    j = new_index()
    k = new_index()
    return t * ξ_[i] * λ_[i] + t**2 / 2 * ξ_[j] * ξ_[k] * λ_[j, k] + t**3 * λ3 + O(t**4)

Then, the above expansions are plugged into the general expression of $\E_{,u}(u, \lambda; \bullet)$ and tested with $\hat{v} \in V$, see Eq. (@eq-20230110092313).

In [26]:
res_v = expand(create_E_u(u_fun=create_u, λ_fun=create_λ) * v_[i]).removeO()

We get an asymptotic expansion of the residual with respect to $\xi_1, \ldots, \xi_m$. This residual must vanish for all $\xi_i$ sufficiently small. Therefore, all coefficients of the expansion `res_v` must be nullified at all orders and for all $\xi_i$ sufficiently small. Considering the expansion up to third order, we get after simplification only *two* bifurcation equations, since the terms of order 0 and 1 are identically null, as verified below.

In [27]:
assert res_v.coeff(t, 0) == 0
assert res_v.coeff(t, 1).subs(E2 * v_[i], 0) == 0

We now consider the terms of order 2 in $\xi_i$ and again standardize the indices: each occurence of $\lambda_a \, \xi_a$ is transformed into $\lambda_j \, \xi_j$, the remaining summation index being renamed $k$.

In [28]:
lhs1 = res_v.coeff(t, 2).expand().subs(E2 * v_[i], 0)

terms = lhs1.args
lhs1 = 0

for term in terms:
    indices = next(iter(get_contraction_structure(term).keys()))
    i1, i2 = indices
    if degree(term, ξ_[i1]) == 1 and degree(term, λ_[i1]) == 1:
        term = term.subs({i1: j, i2: k})
    elif degree(term, ξ_[i2]) == 1 and degree(term, λ_[i2]) == 1:
        term = term.subs({i2: j, i1: k})
    else:
        term = term.subs({i2: j, i1: k})
    lhs1 += term

Eq(lhs1, 0)

Eq(\E_3*\xi[j]*\xi[k]*v[i]*v[j]*v[k]/2 + \dot{\E}_2*\lambda[j]*\xi[j]*\xi[k]*v[i]*v[k], 0)

And, introducing the symbols $\dot{E}_{ij}$ and $E_{ijk}$, defined by Eqs. (@eq-20230125062505) and (@eq-20230125062510), respectively, we get the first equation.

In [29]:
lhs1 = lhs1.subs({
    E3 * v_[i] * v_[j] * v_[k]: E_[i, j, k],
    λ_[j] * ξ_[j] * ξ_[k] * E2_dot * v_[i] * v_[k]: λ_[k] * ξ_[j] * ξ_[k] * E_dot_[i, j]
})

In [30]:
#| code-fold: true
display_latex_equation(lhs1, 0)

<IPython.core.display.Math object>

which also reads

$$
\xi_j \, \xi_k \, \bigl( \tfrac{1}{2} E_{ijk} + \lambda_k \, \dot{E}_{ij} \bigr) = 0,
$$ {#eq-20230115101130}

for all $\xi_1, \ldots, \xi_m \in \reals$.

Eq. (@eq-20230115101130) results from an asymptotic expansion w.r.t. the $\xi_i$. As such it needs only be satisfied for all $\xi_1, \ldots, \xi_m$ *small enough*. However, this equation is homogeneous of degree 2 w.r.t. the $\xi_i$. Therefore, it indeed holds for all $\xi_1, \ldots, \xi_m \in \reals$.

Turning now to the term of third-order in $\xi_i$, the expression with unique indices is **hudge**. Using in each term standard index names ($j$, $k$, $l$) reduces the expression considerably (since many terms that are actually symmetric in these three indices cancel out).

In [31]:
_λ = Idx(r"\lambda")

lhs2 = res_v.coeff(t, 3)
lhs2 = lhs2.subs(E2 * v_[i], 0).expand()

terms = lhs2.args
lhs2 = 0
for term in terms:
    lhs2 += standardize_indices(term)

lhs2 = lhs2.subs(w_is_symmetric).subs(λ_is_symmetric)
    
λl_vj_wkλ = λ_[l] * v_[j] * w_[k, _λ]
vj_wkl = v_[j] * w_[k, l]

lhs2 = lhs2.subs({
    λ_[k, l] * v_[j]: λ_[j, k] * v_[l],
    λ_[j, l] * v_[k]: λ_[j, k] * v_[l],
    λ_[j] * v_[k] * w_[l, _λ]: λl_vj_wkλ,
    λ_[j] * v_[l] * w_[k, _λ]: λl_vj_wkλ,
    λ_[k] * v_[j] * w_[l, _λ]: λl_vj_wkλ,
    λ_[k] * v_[l] * w_[j, _λ]: λl_vj_wkλ,
    v_[k] * w_[j, l]: vj_wkl,
    v_[l] * w_[j, k]: vj_wkl,
})

And we get the second equation  (which is homogeneous of degree 3 w.r.t. the $\xi_i$ and must therefore hold even for large values of these parameters)

In [32]:
#| code-fold: true
display_latex_equation(lhs2, 0)

<IPython.core.display.Math object>

$$
\begin{aligned}[b]
\xi_j \, \xi_k \, \xi_l \, \bigl[
\tfrac{1}{2} \E_3(v_i, v_j, w_{kl})
+ \lambda_l \, \E_3(v_i, v_j, w_{k\lambda})
+ \tfrac{1}{6} \E_4(v_i, v_j, v_k, v_l) &\\
+ \tfrac{1}{2} \lambda_l \, \dot{\E}_2(v_i, w_{jk})
+ \lambda_k \, \lambda_l \, \dot{\E}_2(v_i, w_{j\lambda})
+ \tfrac{1}{2} \lambda_{kl} \, \dot{\E}_2(v_i, v_j) &\\
+ \tfrac{1}{2} \lambda_l \, \dot{\E}_3(v_i, v_j, v_k)
+ \tfrac{1}{2} \lambda_k \, \lambda_l \, \ddot{\E}_2(v_i, v_j)
\bigr] &= 0,
\end{aligned}
$$ {#eq-20230115101147}
for all $\xi_1, \ldots, \xi_m \in \reals$.

## Simplification of the bifurcation equations

In this section, Eqs. (@eq-20230115101130) and (@eq-20230115101147) are simplified to deliver the bifurcation equations (@eq-20230125062336) and (@eq-20230124205642).

### Parametrization of the bifurcated branch

A parametrization of the bifurcated branch is first introduced. The bifurcated branch is a curve $(u, \lambda) \in \reals ^ {m + 1}$, which is parametrized by $\eta$: $[u(\eta), \lambda(\eta)]$, with $u(0) = u_0$ and $\lambda(0) = \lambda_0$; primed quantities denoting derivatives with respect to $\eta$, we introduce

$$
\order[1]{\xi_i} = \xi_i'(0), \quad \order[2]{\xi_i} = \xi_i''(0), \quad \ldots, \quad \order[1]{\lambda} = \lambda'(0), \quad \ldots
$$

In @sec-20230102030125, $\lambda$ was defined as a function of $\xi_1, \ldots, \xi_m$. Therefore, from the chain rule

$$
\lambda'(\eta) = \frac{\partial\lambda}{\partial\xi_i} \, \xi_i'(\eta) \quad \text{and} \quad \lambda''(\eta)  = \frac{\partial^2\lambda}{\partial\xi_i \, \partial\xi_j} \, \xi_i'(\eta) \, \xi_j'(\eta) + \frac{\partial\lambda}{\partial\xi_i} \, \xi_i''(\eta)
$$

and, at $\eta = 0$

$$
\order[1]{\lambda} = \lambda_i \, \order[1]{\xi_i} \quad \text{and} \quad \order[2]{\lambda} = \lambda_{ij} \, \order[1]{\xi_i} \, \order[1]{\xi_j} + \lambda_i \, \order[2]{\xi_i}.
$$ {#eq-20230116205753}

In [33]:
#| code-fold: true

ξ1_ = IndexedBase(r"{\order[1]{\xi}}")
ξ2_ = IndexedBase(r"{\order[2]{\xi}}")
λ1, λ2 = symbols(r"{\order[1]{\lambda}} {\order[2]{\lambda}}")

In [87]:
rules = dict()

rules["λ⁽¹⁾"] = {
    λ_[i] * ξ1_[i]: λ1,
    λ_[j] * ξ1_[j]: λ1,
    λ_[k] * ξ1_[k]: λ1,
    λ_[l] * ξ1_[l]: λ1,
}

rules[r"\dot{E}_ij"] = {
    E2_dot * v_[i] * v_[j]: E_dot_[i, j],
    E2_dot * v_[i] * v_[k]: E_dot_[i, k],
    E2_dot * v_[i] * v_[l]: E_dot_[i, l],
    E2_dot * v_[j] * v_[k]: E_dot_[j, k],
    E2_dot * v_[j] * v_[l]: E_dot_[j, l],
    E2_dot * v_[k] * v_[l]: E_dot_[k, l],
}

rules[r"\ddot{E}_ij"] = {
    E2_ddot * v_[i] * v_[j]: (E_ddot_[i, j]
                              - E2_dot * v_[i] * w_[j, _λ]
                              - E2_dot * v_[j] * w_[i, _λ]),
    E2_ddot * v_[i] * v_[k]: (E_ddot_[i, k]
                              - E2_dot * v_[i] * w_[k, _λ]
                              - E2_dot * v_[k] * w_[i, _λ]),
    E2_ddot * v_[i] * v_[l]: (E_ddot_[i, l]
                              - E2_dot * v_[i] * w_[l, _λ]
                              - E2_dot * v_[l] * w_[i, _λ]),
}

rules[r"E_ijk"] = {
    E3 * v_[i] * v_[j] * v_[k]: E_[i, j, k],
    E3 * v_[i] * v_[j] * v_[l]: E_[i, j, l],
    E3 * v_[i] * v_[k] * v_[l]: E_[i, k, l],
}

rules[r"\dot{E}_ijk"] = {
    E3_dot * v_[i] * v_[j] * v_[k]: (E_dot_[i, j, k] - E2_dot * v_[i] * w_[j, k] 
                                     - E2_dot * v_[j] * w_[i, k] - E2_dot * v_[k] * w_[i, j]),
    E3_dot * v_[i] * v_[j] * v_[l]: (E_dot_[i, j, l] - E2_dot * v_[i] * w_[j, l] 
                                     - E2_dot * v_[j] * w_[i, l] - E2_dot * v_[l] * w_[i, j]),
    E3_dot * v_[i] * v_[k] * v_[l]: (E_dot_[i, k, l] - E2_dot * v_[i] * w_[k, l] 
                                     - E2_dot * v_[k] * w_[i, l] - E2_dot * v_[l] * w_[i, k]),
}

rules["E_ijkl"] = {
    E4 * v_[i] * v_[j] * v_[k] * v_[l]: (E_[i, j, k, l]
                                         - E3 * v_[i] * v_[j] * w_[k, l] 
                                         - E3 * v_[i] * v_[k] * w_[j, l] 
                                         - E3 * v_[i] * v_[l] * w_[j, k])
}

### Simplification of the first bifurcation equation

We first observe that, for fixed $i = 1, \ldots, m$, Eq. (@eq-20230115101130) is of the form $L_{jk} \, \xi_j \, \xi_k = 0$ for all $\xi_1, \ldots, \xi_m$. Derivation with respect to $\xi_l$ leads to

$$
0 = L_{jk} \, \delta_{jl} \, \xi_k + L_{jk} \, \xi_j \, \delta_{kl} = L_{lk} \, \xi_k + L_{jl} \, \xi_j = \bigl(L_{kl} +L_{lk} \bigr) \, \xi_l,
$$

for all $\xi_1, \ldots, \xi_m \in \reals$. The above identity leads to: $L_{kl} + L_{lk} = 0$. Application to
Eq. (@eq-20230115101130) delivers the following identity, to be used below

$$
E_{ijk} + \lambda_k \, \dot{E}_{ij} + \lambda_j \, \dot{E}_{ik} = 0.
$$ {#eq-20230120210008}

Eq. (@eq-20230115101130) holds for all $\xi_1, \ldots, \xi_m$. In particular, it holds for $\xi_i = \order[1]{\xi_i}$ and using
Eq. (@eq-20230116205753) delivers

In [88]:
lhs1a = lhs1.subs(ξ_, ξ1_).subs(rules["λ⁽¹⁾"])

In [89]:
#| code-fold: true
display_latex_equation(lhs1a, 0)

<IPython.core.display.Math object>

which coincides with the **first bifurcation equation** (@eq-20230125062336).

In [90]:
#| code-fold: true
expected = E_[i, j, k] * ξ1_[j] * ξ1_[k] / 2 + λ1 * E_dot_[i, j] * ξ1_[j]

assert lhs1a - expected == 0

### Simplification of the second bifurcation equation

We first recognize $\dot{E}_{ij}$ defined by (@eq-20230125062505) and plug Eq. (@eq-20230116205753)

In [149]:
lhs2a = lhs2.subs(ξ_, ξ1_).subs(rules["λ⁽¹⁾"]).expand()

In [150]:
lhs2a = lhs2.subs(ξ_, ξ1_)
lhs2b = 0
for term in lhs2a.args:
    lhs2b += symmetrize(term)

lhs2b = lhs2b.subs(rules["λ⁽¹⁾"])
names = ["E_ijkl", "E_ijk", "\dot{E}_ijk"]
for name in names:
    lhs2b = lhs2b.subs(rules[name]).expand()
lhs2b

\E_3*{\order[1]{\lambda}}*v[i]*v[j]*w[k, \lambda]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/6 + \E_3*{\order[1]{\lambda}}*v[i]*v[j]*w[l, \lambda]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[l]/6 + \E_3*{\order[1]{\lambda}}*v[i]*v[k]*w[j, \lambda]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/6 + \E_3*{\order[1]{\lambda}}*v[i]*v[k]*w[l, \lambda]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + \E_3*{\order[1]{\lambda}}*v[i]*v[l]*w[j, \lambda]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[l]/6 + \E_3*{\order[1]{\lambda}}*v[i]*v[l]*w[k, \lambda]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 - \E_3*v[i]*v[j]*w[k, l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/12 + \E_3*v[i]*v[j]*w[l, k]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/12 - \E_3*v[i]*v[k]*w[j, l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/12 + \E_3*v[i]*v[k]*w[l, j]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/12 - \E_3*v[i]*v[l]*w[j, k]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/1

The term in $\order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l$ reads

In [151]:
lhs2c = lhs2b.subs({
    E3 * v_[i] * v_[k] * w_[l, _λ]: E2_dot * w_[i, k] * v_[l],
    E3 * v_[i] * v_[l] * w_[k, _λ]: E2_dot * w_[i, l] * v_[k]
}).subs(w_is_symmetric).subs(λ_is_symmetric).expand()

lhs2c

\E_3*{\order[1]{\lambda}}*v[i]*v[j]*w[k, \lambda]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/6 + \E_3*{\order[1]{\lambda}}*v[i]*v[j]*w[l, \lambda]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[l]/6 + \E_3*{\order[1]{\lambda}}*v[i]*v[k]*w[j, \lambda]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/6 + \E_3*{\order[1]{\lambda}}*v[i]*v[l]*w[j, \lambda]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[l]/6 + \ddot{\E}_2*{\order[1]{\lambda}}**2*v[i]*v[j]*{\order[1]{\xi}}[j]/6 + \ddot{\E}_2*{\order[1]{\lambda}}**2*v[i]*v[k]*{\order[1]{\xi}}[k]/6 + \ddot{\E}_2*{\order[1]{\lambda}}**2*v[i]*v[l]*{\order[1]{\xi}}[l]/6 + \dot{\E}_2*{\order[1]{\lambda}}**2*v[i]*w[j, \lambda]*{\order[1]{\xi}}[j]/3 + \dot{\E}_2*{\order[1]{\lambda}}**2*v[i]*w[k, \lambda]*{\order[1]{\xi}}[k]/3 + \dot{\E}_2*{\order[1]{\lambda}}**2*v[i]*w[l, \lambda]*{\order[1]{\xi}}[l]/3 - \dot{\E}_2*{\order[1]{\lambda}}*v[j]*w[i, k]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/6 - \dot{\E}_2*{\order[1]{\lambda}}*v[j]*w[i, l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[l]/6 - \d

The variational problems (@eq-20230107180410) and (@eq-20230107180501) are used to transform $\E_3(v_i, v_k, w_{k\lambda})$ and $\E_3(v_i, v_l, w_{k\lambda})$ as follows

$$
\E3(v_i, v_j, w_{k\lambda}) = -\E_2(w_{ij}, w_{k\lambda}) = \dot{\E}_2(w_{ij}, v_k)
$$

In [152]:
lhs2d = lhs2c.subs({
    E3 * v_[i] * v_[j] * w_[k, _λ]: E2_dot * w_[i, k] * v_[j],
    E3 * v_[i] * v_[k] * w_[j, _λ]: E2_dot * w_[i, j] * v_[k],
    E3 * v_[i] * v_[k] * w_[l, _λ]: E2_dot * w_[i, k] * v_[l],
    E3 * v_[i] * v_[l] * w_[k, _λ]: E2_dot * w_[i, l] * v_[k],
    E3 * v_[i] * v_[j] * w_[l, _λ]: E2_dot * w_[i, l] * v_[j],
    E3 * v_[i] * v_[l] * w_[j, _λ]: E2_dot * w_[i, j] * v_[l],
}).expand()

lhs2d

\ddot{\E}_2*{\order[1]{\lambda}}**2*v[i]*v[j]*{\order[1]{\xi}}[j]/6 + \ddot{\E}_2*{\order[1]{\lambda}}**2*v[i]*v[k]*{\order[1]{\xi}}[k]/6 + \ddot{\E}_2*{\order[1]{\lambda}}**2*v[i]*v[l]*{\order[1]{\xi}}[l]/6 + \dot{\E}_2*{\order[1]{\lambda}}**2*v[i]*w[j, \lambda]*{\order[1]{\xi}}[j]/3 + \dot{\E}_2*{\order[1]{\lambda}}**2*v[i]*w[k, \lambda]*{\order[1]{\xi}}[k]/3 + \dot{\E}_2*{\order[1]{\lambda}}**2*v[i]*w[l, \lambda]*{\order[1]{\xi}}[l]/3 + \dot{\E}_2*\lambda[j, k]*v[i]*v[l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + \dot{\E}_2*\lambda[j, l]*v[i]*v[k]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + \dot{\E}_2*\lambda[k, l]*v[i]*v[j]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + {\order[1]{\lambda}}*\dot{E}[i, j, k]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/6 + {\order[1]{\lambda}}*\dot{E}[i, j, l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[l]/6 + {\order[1]{\lambda}}*\dot{E}[i, k, l]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + E[i, j, k, l]

$$
2\dot{\E}_2(v_i, w_{l\lambda})
= \dot{\E}_2(v_i, w_{l\lambda}) -\E_2(w_{i\lambda}, w_{l\lambda})
= \dot{\E}_2(v_i, w_{l\lambda}) + \E_2(v_l, w_{i\lambda})
= \ddot{E}_{il} - \ddot{E}_2(v_i, v_l),
$$

where $\ddot{E}_{ij}$ is defined by Eq. (@eq-20230124211207). 

In [153]:
lhs2e = lhs2d.subs({
    E2_dot * v_[i] * w_[j, _λ]: (E_ddot_[i, j] - E2_ddot * v_[i] * v_[j]) / 2,
    E2_dot * v_[i] * w_[k, _λ]: (E_ddot_[i, k] - E2_ddot * v_[i] * v_[k]) / 2,
    E2_dot * v_[i] * w_[l, _λ]: (E_ddot_[i, l] - E2_ddot * v_[i] * v_[l]) / 2,
}).subs(rules[r"\ddot{E}_ij"]).expand()
lhs2e

\dot{\E}_2*\lambda[j, k]*v[i]*v[l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + \dot{\E}_2*\lambda[j, l]*v[i]*v[k]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + \dot{\E}_2*\lambda[k, l]*v[i]*v[j]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + {\order[1]{\lambda}}**2*\ddot{E}[i, j]*{\order[1]{\xi}}[j]/6 + {\order[1]{\lambda}}**2*\ddot{E}[i, k]*{\order[1]{\xi}}[k]/6 + {\order[1]{\lambda}}**2*\ddot{E}[i, l]*{\order[1]{\xi}}[l]/6 + {\order[1]{\lambda}}*\dot{E}[i, j, k]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/6 + {\order[1]{\lambda}}*\dot{E}[i, j, l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[l]/6 + {\order[1]{\lambda}}*\dot{E}[i, k, l]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + E[i, j, k, l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6

Upon susbtitution of $\order[1]{\xi}_j \, \order[1]{\xi}_k \, \lambda_{jk}$ with $\order[2]{\lambda} - \lambda_k \, \order[2]{\xi_k}$ \[see Eq. (@eq-20230116205753)\]

In [165]:
(lhs2f := lhs2e.subs({
    ξ1_[j] * ξ1_[k] * λ_[j, k]: λ2 - λ_[j] * ξ2_[j],
    ξ1_[j] * ξ1_[l] * λ_[j, l]: λ2 - λ_[j] * ξ2_[j],
    ξ1_[k] * ξ1_[l] * λ_[k, l]: λ2 - λ_[k] * ξ2_[k]
}).subs(rules[r"\dot{E}_ij"]).expand())

{\order[1]{\lambda}}**2*\ddot{E}[i, j]*{\order[1]{\xi}}[j]/6 + {\order[1]{\lambda}}**2*\ddot{E}[i, k]*{\order[1]{\xi}}[k]/6 + {\order[1]{\lambda}}**2*\ddot{E}[i, l]*{\order[1]{\xi}}[l]/6 + {\order[1]{\lambda}}*\dot{E}[i, j, k]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/6 + {\order[1]{\lambda}}*\dot{E}[i, j, l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[l]/6 + {\order[1]{\lambda}}*\dot{E}[i, k, l]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + {\order[2]{\lambda}}*\dot{E}[i, j]*{\order[1]{\xi}}[j]/6 + {\order[2]{\lambda}}*\dot{E}[i, k]*{\order[1]{\xi}}[k]/6 + {\order[2]{\lambda}}*\dot{E}[i, l]*{\order[1]{\xi}}[l]/6 + E[i, j, k, l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 - \dot{E}[i, j]*\lambda[k]*{\order[1]{\xi}}[j]*{\order[2]{\xi}}[k]/6 - \dot{E}[i, k]*\lambda[j]*{\order[1]{\xi}}[k]*{\order[2]{\xi}}[j]/6 - \dot{E}[i, l]*\lambda[j]*{\order[1]{\xi}}[l]*{\order[2]{\xi}}[j]/6

Finally, we use Eq. (@eq-20230120210008)

In [166]:
(lhs2g := lhs2f.subs({
    λ_[j] * ξ2_[j] * E_dot_[i, k]: -(E_[i, j, k] + λ_[k] * E_dot_[i, j]) * ξ2_[j],
    λ_[j] * ξ2_[j] * E_dot_[i, l]: -(E_[i, j, l] + λ_[l] * E_dot_[i, j]) * ξ2_[j],
    λ_[k] * ξ2_[k] * E_dot_[i, j]: -(E_[i, j, k] + λ_[j] * E_dot_[i, k]) * ξ2_[k],
    λ_[k] * ξ2_[k] * E_dot_[i, l]: -(E_[i, k, l] + λ_[l] * E_dot_[i, k]) * ξ2_[k],
    λ_[l] * ξ2_[l] * E_dot_[i, j]: -(E_[i, j, l] + λ_[j] * E_dot_[i, l]) * ξ2_[l],
    λ_[l] * ξ2_[l] * E_dot_[i, k]: -(E_[i, k, l] + λ_[k] * E_dot_[i, l]) * ξ2_[l],
}).subs(rules["λ⁽¹⁾"]).expand())

{\order[1]{\lambda}}**2*\ddot{E}[i, j]*{\order[1]{\xi}}[j]/6 + {\order[1]{\lambda}}**2*\ddot{E}[i, k]*{\order[1]{\xi}}[k]/6 + {\order[1]{\lambda}}**2*\ddot{E}[i, l]*{\order[1]{\xi}}[l]/6 + {\order[1]{\lambda}}*\dot{E}[i, j, k]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/6 + {\order[1]{\lambda}}*\dot{E}[i, j, l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[l]/6 + {\order[1]{\lambda}}*\dot{E}[i, k, l]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + {\order[2]{\lambda}}*\dot{E}[i, j]*{\order[1]{\xi}}[j]/6 + {\order[2]{\lambda}}*\dot{E}[i, k]*{\order[1]{\xi}}[k]/6 + {\order[2]{\lambda}}*\dot{E}[i, l]*{\order[1]{\xi}}[l]/6 + E[i, j, k, l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + E[i, j, k]*{\order[1]{\xi}}[j]*{\order[2]{\xi}}[k]/6 + E[i, j, k]*{\order[1]{\xi}}[k]*{\order[2]{\xi}}[j]/6 + E[i, j, l]*{\order[1]{\xi}}[l]*{\order[2]{\xi}}[j]/6 + \dot{E}[i, j]*\lambda[k]*{\order[1]{\xi}}[k]*{\order[2]{\xi}}[j]/6 + \dot{E}[i, j]*\lambda[l]*{\order[1]{\xi}}[l]*{\order[2]{\xi}}[j]/6 + \dot{E}

In [167]:
(lhs2h := lhs2g.subs({
    E_dot_[i, k] * ξ1_[k] : E_dot_[i, j] * ξ1_[j],
    E_dot_[i, l] * ξ1_[l] : E_dot_[i, j] * ξ1_[j],
    E_dot_[i, k] * ξ2_[k] : E_dot_[i, j] * ξ1_[j],
    E_dot_[i, l] * ξ2_[l] : E_dot_[i, j] * ξ1_[j],
    E_ddot_[i, k] * ξ1_[k] : E_ddot_[i, j] * ξ1_[j],
    E_ddot_[i, l] * ξ1_[l] : E_ddot_[i, j] * ξ1_[j],
    E_[i, j, k] * ξ1_[k] * ξ2_[j]: E_[i, j, k] * ξ1_[j] * ξ2_[k],
    E_[i, j, l] * ξ1_[j] * ξ2_[l]: E_[i, j, k] * ξ1_[j] * ξ2_[k],
    E_[i, j, l] * ξ1_[l] * ξ2_[j]: E_[i, j, k] * ξ1_[j] * ξ2_[k],
    E_[i, k, l] * ξ1_[k] * ξ2_[l]: E_[i, j, k] * ξ1_[j] * ξ2_[k],
    E_[i, k, l] * ξ1_[l] * ξ2_[k]: E_[i, j, k] * ξ1_[j] * ξ2_[k],
    E_dot_[i, j, l] * ξ1_[j] * ξ1_[l]: E_dot_[i, j, k] * ξ1_[j] * ξ1_[k],
    E_dot_[i, k, l] * ξ1_[k] * ξ1_[l]: E_dot_[i, j, k] * ξ1_[j] * ξ1_[k],
}).expand().subs(rules["λ⁽¹⁾"]))

{\order[1]{\lambda}}**2*\ddot{E}[i, j]*{\order[1]{\xi}}[j]/2 + {\order[1]{\lambda}}*\dot{E}[i, j, k]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/2 + {\order[1]{\lambda}}*\dot{E}[i, j]*{\order[1]{\xi}}[j]/6 + {\order[1]{\lambda}}*\dot{E}[i, j]*{\order[2]{\xi}}[j]/3 + {\order[2]{\lambda}}*\dot{E}[i, j]*{\order[1]{\xi}}[j]/2 + E[i, j, k, l]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/6 + E[i, j, k]*{\order[1]{\xi}}[j]*{\order[2]{\xi}}[k]/2

and Eq. (@eq-20230124205642) is retrieved.

## Taylor expansion of the bifurcated branch

We now turn to $w$, which was defined in @sec-20230102030010 as an implicit function of $\xi_1, \ldots, \xi_m$ and
$\lambda$. It is now defined as a function of the arc parameter $\eta$ as follows: $w(n) = w[\xi_1(\eta), \ldots, \xi_m(\eta), \lambda(\eta)]$.
From the chain rule
$$
w'(\eta) = \frac{\partial w}{\partial \xi_i} \, \xi_i' + \frac{\partial w}{\partial \lambda} \, \lambda'
$$
and
$$
w''(\eta) = \frac{\partial^2 w}{\partial \xi_i \, \partial \xi_j} \, \xi_i' \, \xi_j'
+ 2\frac{\partial w}{\partial \xi_i \, \partial \lambda} \, \xi_i' \, \lambda'
+ \frac{\partial w}{\partial \xi_i} \, \xi_i''
+ \frac{\partial^2 w}{\partial \lambda^2} \, \lambda^{'2}
+ \frac{\partial w}{\partial \lambda} \, \lambda''.
$$

At $\eta = 0$, the above identities reduce to
$$
w'(0) = 0
\quad \text{and} \quad
w''(0) = \order[1]{\xi_i} \, \order[1]{\xi_j} \, w_{ij}  + 2 \order[1]{\lambda} \, \order[1]{\xi_i} \, w_{i\lambda}
$$

We deduce from the above the Taylor expansion (@eq-20230528174310) of the bifurcated branch as $\eta \to 0$.